In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


# Creating a General Sequential model
###  prerequisite
####  the column which is to predicted should be at the end,the model will predict the content in the last column,dataset should be free from all the meaningless data such as "id"

In [2]:
dataset=pd.read_csv("heart.csv")    #diabetes.csv can be any dataset
columns=dataset.columns          #getting the names of all the columns in dataset
columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [3]:
def one_hot_encode(column):
    list=[]                   #creating a list that will hold the encoded column
    unq=column.unique()       #getting all the unique values in column
    n=column.nunique()        #getting the number of unique values in columns
    for i in column:          #taking each value in column
        temp=[0 for k in range(n)]     #for storing the value of option i.e. [0,0,0,.....,upto n]
        for j in range(len(unq)):   #accesing the position of each element in unq list
            if i==unq[j]:           #if the value of column is same as the value at unq[j]
                temp[j]=1           # then   [0,0,..,1(at jth location),....,upto n]
                list.append(temp)   #adding it to the one hot encoded column 
                break
    return list                #returning the list

In [4]:
mdataset=[]
for i in columns[:-1]:  #each column in dataset
    flag=0
    column1=dataset[i].unique()#unique values of each column
    for j in column1:
        if type(j)==str: #if any of the unique value is a string
            mdataset+=[one_hot_encode(dataset[i])]#then before adding one hot encoding is done
            flag=1           # to check if the column is already added
            break
    if flag==0:      #if the column is not added means it does not have any string in it and add the column directly
        temp=[[i] for i in dataset[i]]  #without the heading      # mdataset+=dataset[i] also adds the heading 
        mdataset+=[temp]
#here mdataset is [[column1,.......],[column2........],[column3......]]
'''
but we want mdataset to be 
[  [column1[0],column2[0],....],
   [column1[1],column2[1],....],
   [column1[2],column2[2],....],]'''

'\nbut we want mdataset to be \n[  [column1[0],column2[0],....],\n   [column1[1],column2[1],....],\n   [column1[2],column2[2],....],]'

In [5]:
final=one_hot_encode(dataset[columns[-1]])        #storing the result(last column) in final

In [6]:
modataset=[]                    #converting the dataset from 1D to required dimension
for i in range(len(dataset[columns[0]])):  #i varies form 0 to length of a first column - 1
    temp=[j[i] for j in mdataset]       # creating a temp list that holds the ith element of each element of mdataset 
    # now temp has list in list i.e. [ [..],[..],[...],[...], ] but we want [ [..,..,..,..,]]  
    row=[]                           #adding all the elements of temp  
    for j in temp:
        row+=j
    modataset.append(row)    #adding a row in modataset
temp

[[57], [0, 1], [1], [130], [236], [0], [0], [174], [0], [0.0], [1], [1], [2]]

In [7]:
in_di=len(modataset[0])    #input dimesion is same as length of any row in modataset
ou_di=len(final[0])        #output dimension is same as length of any row in final
print(in_di,ou_di)
mdataset=np.array(modataset)
mfinal=np.array(final)

14 2


In [8]:
x_train,x_test,y_train,y_test=train_test_split(mdataset,mfinal,test_size=0.2,random_state=35)

In [9]:
#building the model,feeding dataset to the model,training,testing...
model=Sequential()
model.add(Dense(500,input_dim=in_di,activation="sigmoid"))
model.add(Dense(100,activation="sigmoid"))
model.add(Dense(ou_di,activation="softmax"))
model.compile(loss="mean_squared_error",optimizer='adam',metrics=["accuracy"])
model.fit(x_train,y_train,epochs=500,batch_size=50,validation_data=(x_test,y_test))

Train on 242 samples, validate on 61 samples
Epoch 1/500
242/242 [==============================] - 0s 2ms/step - loss: 0.2671 - acc: 0.4504 - val_loss: 0.2335 - val_acc: 0.5738
Epoch 2/500
242/242 [==============================] - 0s 74us/step - loss: 0.2325 - acc: 0.6033 - val_loss: 0.2264 - val_acc: 0.6721
Epoch 3/500
242/242 [==============================] - 0s 83us/step - loss: 0.2211 - acc: 0.6612 - val_loss: 0.2086 - val_acc: 0.6066
Epoch 4/500
242/242 [==============================] - 0s 78us/step - loss: 0.2106 - acc: 0.6653 - val_loss: 0.2061 - val_acc: 0.7049
Epoch 5/500
242/242 [==============================] - 0s 92us/step - loss: 0.2042 - acc: 0.6942 - val_loss: 0.2082 - val_acc: 0.7049
Epoch 6/500
242/242 [==============================] - 0s 87us/step - loss: 0.2023 - acc: 0.6942 - val_loss: 0.2081 - val_acc: 0.6721
Epoch 7/500
242/242 [==============================] - 0s 74us/step - loss: 0.1984 - acc: 0.6983 - val_loss: 0.2085 - val_acc: 0.6721
Epoch 8/500
242/24

In [10]:
 # given a string and the NAME of the column in which it exits returning the one hot encoding it originally had
def get_1_0(string,column):              
    unq=dataset[column].unique()         #unq holds all unique value in that column
    temp=[0 for k in range(len(unq))]   # [0,0,0,.....,upto len(unq)]
    for i in range(len(unq)):         # if the entered string is same as the element unq[i] 
        if unq[i]==string:            #   [0,0,0,.....1(at location i)]
            temp[i]=1
    return temp

In [12]:
def get_input():    # get all the inputs
    inp=[]           # to store input          
    for j in columns[0:len(columns)-1]:   #j holds the column NAME except the  last 1
        flag=0 # flag to check if the value to be entered is a string , int , float
        unq=dataset[j].unique()           
        for i in unq:          # if any of the values in column has string 
            if type(i)==str:   # we need to print "enter (column name) in (all unique strings in the column)"    
                unqstr="("
                for k in unq:
                    unqstr+=k+","
                unqstr+="): "
                string=input("enter "+j+" in "+unqstr)
                inp+=get_1_0(string,j)  
                flag="string"
                break
        for i in unq:
            if type(i)==int:
                inp+=[int(input("enter "+j+" : "))]
                flag="int"
        if flag==0:
            inp+=[float(input("enter "+j+" : "))]
    return inp
inp=get_input()

enter age : 63
enter sex in (Male,Female,): Male
enter cp : 3
enter trestbps : 145
enter chol : 233
enter fbs : 1
enter restecg : 0
enter thalach : 150
enter exang : 0
enter oldpeak : 2.3
enter slope : 0
enter ca : 0
enter thal : 1


In [13]:
result=model.predict_classes([[inp]])
unq=dataset[columns[-1]]  # all unique values in last column
for i in range(ou_di):      
    if result==i:          
        print(unq[i])

greater than 50% narrowing


In [14]:
#63,Male,3,145,233,1,0,150,0,2.3,0,0,1